In [1]:
# For API requests
# For API requests
import requests

# For managing json files
import json

# For processing data
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta
import matplotlib.dates as mdates

import plotly.express as px
import plotly.graph_objects as go

In [ ]:


url = 'https://data.cityofnewyork.us/resource/h9gi-nx95.json?' 
token = 'c7nn42f3kh67innzws0qb33hd' # personal token

def GetData(url,token):
    hed = {'X-Auth-Token': f'{token}'}
    url = url + '$limit=2000000' # Adding limit 2.000.000 to get all the data
    response = requests.get(url, headers=hed)
    response_jason = json.loads(response.text)
    df = pd.json_normalize(response_jason)
    return df

df = GetData(url,token)

In [ ]:
df.to_csv("Motor_Vehicle_Collisions-Crashes1.csv")

In [2]:
file = 'Motor_Vehicle_Collisions-Crashes1.csv'
df = pd.read_csv(file)

C:\Users\Julian\AppData\Local\Temp\ipykernel_6824\121783900.py:2: DtypeWarning: Columns (18,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [7]:
df.columns

Index(['Unnamed: 0', 'crash_date', 'crash_time', 'on_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'collision_id', 'vehicle_type_code1', 'vehicle_type_code2', 'borough',
       'zip_code', 'latitude', 'longitude', 'off_street_name',
       'location.latitude', 'location.longitude', 'location.human_address',
       'contributing_factor_vehicle_3', 'vehicle_type_code_3',
       'cross_street_name', 'contributing_factor_vehicle_4',
       'vehicle_type_code_4', 'contributing_factor_vehicle_5',
       'vehicle_type_code_5'],
      dtype='object')

In [9]:

#ETL
# me quedo con las columnas que voy a utilizar
df.drop(df.columns.difference(['crash_date','number_of_persons_injured','number_of_persons_killed']), axis=1, inplace=True)
# ordeno y transfrmo campo date
df['crash_date']=pd.to_datetime(df['crash_date'])
df = df.sort_values('crash_date',ascending=True)

In [3]:
# hago campo total país, para introducción del análisis
# agrupo por fecha
df_evol1 = df.groupby('crash_date',as_index=False)[['number_of_persons_injured','number_of_persons_killed']].sum()
# ordeno por fecha
df_evol1 = df_evol1.sort_values('crash_date',ascending=True)

In [5]:
# Grafico introductorio, ocupación de camas, camas ICU y acumulado de muertes a TOTAL PAIS.

fig = px.line(df_evol1,x = df_evol1['crash_date'],
                y=['number_of_persons_injured','number_of_persons_killed'], 
                title='Evolución siniestros NYC', width=800, height=550,
                labels= {'date':'Fechas','value':'Cantidad'})

fig.add_trace(
    go.Scatter(x=list(df_evol1.crash_date), y=list(df_evol1['number_of_persons_injured'])
                , name='Cantidad de personas lastimadas'))
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()